In [1]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict, namedtuple
from tqdm import tqdm
from unidecode import unidecode
import os
import pickle as pk
import json
import random
import string
from tqdm import tqdm

In [2]:
with open('metadata.txt', 'r') as f:
    start_page = int(f.read()) + 1
start_page

2001

In [3]:
def id_generator(size=10, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [4]:
def extract(url):
    try:
        content = requests.get(url).content
        soup = BeautifulSoup(content, "html.parser")
        title = soup.find_all('h1', {'class': 'first-title'})[0].text
        text = ' '.join([p.text for p in soup.find_all('div', {'itemprop': 'articleBody'})])
        tags = soup.find_all('footer', {'class': 'tags'})[0].find_all('ul')[0].text.split('\n')
        tags = [tag for tag in tags if tag != '']
        category = soup.find_all('div', {'class': 'service-title'})[0].text.replace('\n', '').split()[1]
        num_comment = 0
    except:
        title, text, tags, category, num_comment = '', '', '', [], 0
    comments = []
    return title, text, tags, category, num_comment, comments

In [5]:
def write_file(dirname, link, title, text, tags, category, num_comment, comments):
    data = {
        'link': link,
        'title': title,
        'text': text,
        'tags': tags,
        'category': category,
        'num_comment': num_comment,
        'comments': comments
    }
    filename = os.path.join(dirname, id_generator())
    with open(os.path.join(filename), 'w', encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False)

In [6]:
A = [i for i in range(1, 20000)]
random.seed(10000)
random.shuffle(A)

In [7]:
links = []
for idx, i in tqdm(enumerate(A[start_page: ])):
    isna_main = requests.get(f'https://www.isna.ir/page/search.xhtml?dt=1401%2F12%2F27&q=%D8%A7%DB%8C%D8%B3%D9%86%D8%A7&a=0&df=&alltp=true&allpl=true&pi={i}&allsr=true&pageSize=20&sort=date&dr=all&allty=true').content
    soup = BeautifulSoup(isna_main, "html.parser")
    link_section = soup.find_all('div', {'class': 'items'})[0]
    news_links = link_section.find_all('li')
    news_links = ['https://www.isna.ir' + li.find_all('a')[0]['href'] for li in link_section.find_all('li')]
    
    for link in news_links:
        links.append(link)
        text, title, tags, category, num_comment, comments = extract(link)
        write_file('news', link, title, text, tags, category, num_comment, comments)
    with open('metadata.txt', 'w', encoding="utf-8") as f:
        f.write(str(start_page + idx))

80it [2:08:19, 79.08s/it] 